# Apply Bucketing

In vanilla quantization, normalization may introduce high variance.

In [1]:
import numpy as np
import torch
import math
import numpy.linalg as LA

In [2]:
def encode(v, bucket_size=512, **kwargs):
    if isinstance(v, (torch.Tensor, torch.cuda.FloatTensor)):
        w = v.cpu().numpy().flat[:]
    elif isinstance(v, np.ndarray):
        w = v.flat[:]
    else:
        raise ValueError("Object passed to encode not ndarray or torch.Tensor")
        
    # Apply bucketing
    if bucket_size != 0:
        code_buckets = []
        shape = v.shape
        print 'shape\t', shape
        
        buckets = np.split(w, (w.shape[0]+bucket_size-1) / bucket_size)
        for bucket in buckets:
            code = encode(bucket, bucket_size=0, **kwargs)
            code_buckets.append(code)
        return {'code_buckets': code_buckets, 'shape': shape}

    norm = LA.norm(v)

    quantization_level = kwargs['quantization_level']
    s = (1 << quantization_level) - 1
    shape = v.shape
    num_int_each_64_bits = 64 / (2 + quantization_level)
    num_section = num_int_each_64_bits
    len_each_section = (w.shape[0] + num_section - 1) / num_section
    w = np.pad(w, (0, len_each_section*num_section - w.shape[0]), mode='constant')

    sign_array = np.sign(w)
    sign_array += 1
    sign_array = sign_array.astype('uint64')
    normalization_array = np.abs(w) / norm * s

    truncated_array = normalization_array.astype(int)
    prob_array =  normalization_array - truncated_array
    dice_array = np.random.rand(len(prob_array))
    xi_array = truncated_array + (dice_array > prob_array)
    xi_array = xi_array.astype('uint64')
    
    old_sign_array = sign_array
    old_xi_array = xi_array
    
    xi_array = xi_array.reshape((num_section, len_each_section))
    sign_array = sign_array.reshape((num_section, len_each_section))
    
    neo_array = np.zeros(len_each_section)
    neo_array = neo_array.astype('uint64')

    for i in range(num_int_each_64_bits):
        xi = xi_array[i]
        sign = sign_array[i]
        neo_array <<= (2 + quantization_level)
        neo_array = neo_array | (sign << quantization_level | xi)

    code = {'neo': neo_array, 'norm': norm, 'quantization_level': quantization_level,
            'len_each_section': len_each_section, 'num_int_each_64_bits': num_int_each_64_bits,
            'shape': shape}

    return code

In [3]:
def decode(code, bucket_size=512, cuda=False, implementation='numpy', codes=[], **kwargs):
    if implementation == 'numpy':
        # Decode from bucketing
        if bucket_size != 0:
            v_list = []
            for code_bucket in code['code_buckets']:
                v = decode(code=code_bucket, bucket_size=0,
                           cuda=cuda, implementation=implementation,
                           codes=codes, **kwargs)
                v_list.extend(v)
            v = np.array(v_list)
            v = v.reshape(code['shape'])
        else:
            norm = code['norm']
            quantization_level = code['quantization_level']
            s = (1 << quantization_level) - 1

            real_size = np.prod(code['shape'])

            neo_array = code['neo'].astype('uint64')
            num_int_each_64_bits = code['num_int_each_64_bits']
            num_section = num_int_each_64_bits
            len_each_section = code['len_each_section']
            xi_array = np.ones((num_section, len_each_section))
            sign_array = np.ones((num_section, len_each_section))
            mask_for_xi = (1 << quantization_level) - 1
            mask_for_sign = 3 << quantization_level
            for i in range(num_int_each_64_bits)[::-1]:
                sign_array[i] = (neo_array & mask_for_sign) >> quantization_level
                xi_array[i] = neo_array & mask_for_xi
                neo_array >>= (2 + quantization_level)

            xi_array = xi_array.reshape(-1).astype('uint64')
            sign_array = sign_array.reshape(-1).astype('int8')
            sign_array -= 1
            v = sign_array * xi_array * norm / s

            v = v[:real_size]
            v = v.reshape(code['shape'])
    else:
        raise ValueError('Whoops, implementation')
    return v

In [4]:
test_a = np.random.rand(5, 4)

print 'each size: {}, total size: {}'.format(test_a[0].nbytes, test_a.nbytes)
print
print 'Original array:'
print test_a

each size: 32, total size: 160

Original array:
[[ 0.61673933  0.86467965  0.25606733  0.35983486]
 [ 0.8342569   0.04360524  0.5447879   0.77115447]
 [ 0.78620767  0.27697439  0.03115012  0.92166798]
 [ 0.7214374   0.56996659  0.00916378  0.02055269]
 [ 0.06300408  0.12190967  0.82373013  0.13947622]]


In [5]:
bucket_size = 6
for quantization_level in range(1, 10):
    print 'Quantization level: {}'.format(quantization_level)
    kwargs = {'quantization_level': quantization_level}
    code = encode(test_a, bucket_size=bucket_size, **kwargs)
    v = decode(code=code, bucket_size=bucket_size)
    print v
    print
    print
    print

Quantization level: 1
shape	(5, 4)
[[ 0.          1.4209414   0.          1.4209414 ]
 [ 1.4209414   1.26024276  0.          0.        ]
 [ 0.          1.26024276  1.3022519   0.        ]
 [ 0.          1.3022519   1.3022519   0.84689954]
 [ 0.          0.84689954  0.          0.84689954]]



Quantization level: 2
shape	(5, 4)
[[ 0.47364713  0.47364713  0.47364713  0.        ]
 [ 0.94729427  0.42008092  0.84016184  0.42008092]
 [ 0.42008092  0.          0.43408397  1.3022519 ]
 [ 0.86816793  0.86816793  0.43408397  0.28229985]
 [ 0.28229985  0.28229985  0.56459969  0.28229985]]



Quantization level: 3
shape	(5, 4)
[[ 0.81196651  1.01495814  0.20299163  0.20299163]
 [ 1.01495814  0.18003468  0.72013872  0.72013872]
 [ 0.9001734   0.36006936  0.18603599  0.74414394]
 [ 0.55810796  0.74414394  0.18603599  0.        ]
 [ 0.12098565  0.2419713   0.72591389  0.2419713 ]]



Quantization level: 4
shape	(5, 4)
[[ 0.56837656  0.94729427  0.18945885  0.28418828]
 [ 0.75783541  0.          0.588